In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(options=options)
    return driver

def process_url(url):
    driver = web_driver()
    try:
        driver.get(url)
        time.sleep(5)

        # Click on the element to expand the description
        xpath_expression = "/html/body/div/main/div[2]/div/div[3]/div[3]/div[1]/div[1]/div/div[3]/p/span"
        element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath_expression)))
        element.click()

        # Wait for the description element to be visible
        xpath_description = '//*[@id="company-container"]/div[3]/div[3]/div[1]/div[1]/div/div[3]/p'
        description_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, xpath_description)))

        # Extract the company description
        description_text = description_element.text.strip()

        # Extract the link URL
        link_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "About_text__mxRe_")))
        link_url = link_element.get_attribute("href")

        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        company_name_element = soup.find('h1', class_='header_title__Zb2Pc')
        if company_name_element:
            company_name = company_name_element.text.strip()
        else:
            company_name = None
        # Extract industry, funding, and people information based on href attributes
        industry_element = soup.find('a', href=lambda href: href and href.startswith("/industries"))
        funding_element = soup.find('a', href=lambda href: href and href.startswith("/companies/funding-sources/"))
        people_element = soup.find('a', href=lambda href: href and href.startswith("/companies/sizes/"))

        # Extract the text content
        industry = industry_element.text.strip() if industry_element else None
        funding = funding_element.text.strip() if funding_element else None
        people = people_element.text.strip() if people_element else None

        return {

            "repvue_url": url,
            "company_url": link_url,
            "Company Name": company_name,
            "Description": description_text,
            "employee_count": people,
            "Funding": funding,
            "Industry": industry,
             }

    except Exception as e:
        print(f"An error occurred for URL '{url}': {e}")
        return None

    finally:
        driver.quit()

# Read URLs from Excel file
df_urls = pd.read_excel('/content/Book4.xlsx', sheet_name='Sheet1', engine='openpyxl')  # Replace 'your_file.xlsx' with your Excel file path and sheet name

# Extract URLs from a specific column in the DataFrame
url_list = df_urls['url'].tolist()

# Process each URL and store the results
results = []
for url in url_list:
    result = process_url(url)
    if result:
        results.append(result)

# Create a DataFrame from the results
df_results = pd.DataFrame(results)

# Save the DataFrame to a new Excel file
output_file = 'extracted_data.xlsx'
df_results.to_excel(output_file, index=False)

# Display the DataFrame
print(df_results)

# Display the path to the output file
print(f"Data has been extracted and saved to '{output_file}'.")


In [ ]:
import pandas as pd

# Load the extracted data from the Excel file
input_file = 'extracted_data.xlsx'
df = pd.read_excel(input_file)

# Clean up the description format by removing extra spaces and line breaks
df['Description'] = df['Description'].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)

# Save the cleaned DataFrame back to the Excel file
output_file_cleaned = 'extracted_data_cleaned.xlsx'
df.to_excel(output_file_cleaned, index=False)

# Display the path to the cleaned output file
print(f"Cleaned data has been saved to '{output_file_cleaned}'.")

# Display the first few rows of the cleaned DataFrame
print(df.head())
